# SEIR model parameter determination from Re estimation

In [1]:
import numpy as np
import pandas as pd
import toml
import matplotlib.pyplot as plt
import time
from datetime import datetime
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator

import epyestim
import epyestim.covid19 as covid19
from epyestim.distributions import discretise_gamma
from datetime import timedelta

# rpy2 imports
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.robjects as robjects
pandas2ri.activate()
# R library
global eps
eps = importr("EpiEstim")
import sys
from pathlib import Path
sys.path.insert(1, '../')
from src2.models.cv19sim import cv19sim

/home/samuel/anaconda3/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [2]:
#Pop-up plots
import platform
OS = platform.system()
if OS == 'Linux' or OS == 'Darwin':
    %matplotlib tk    
elif OS == 'Windows':
    %matplotlib qt    

## Bassi A. Equations

In [164]:
def getdelta(y,x=None,scale='linear'):
    if scale == 'linear':
        y = np.log(y+0.001)
    #delta = np.diff(y)
    delta = np.gradient(y)
    return delta

def delta_eq(beta,S,N,sigma,gamma):
    delta = 1/2*(np.sqrt(4*sigma*beta*S/N+(sigma-gamma)**2)-sigma-gamma)
    return delta
def Re_delta_calc(delta,sigma,gamma):
    Ti = 1/gamma
    Te = 1/sigma
    Ts = Ti + Te
    return 1 + delta*Ts + sigma*gamma/((sigma+gamma)**2)*(delta*Ts)**2 

def Re_delta_a_calc(delta,sigma,gamma):
    Ti = 1/gamma
    Te = 1/sigma
    Ts = Ti + Te
    return 1 + delta*Ts

def Re_analitico_calc(beta,gamma,S,N):
    Re = beta/gamma*S/N
    return Re

def beta_Re(Re,S,N,gamma):
    return Re*gamma*N/S

def beta_delta(delta,S,N,gamma,sigma):
    Re = Re_delta_calc(delta,sigma,gamma)
    return Re*gamma*N/S

def beta_Id(I_d,S,N,gamma,sigma):
    delta = getdelta(I_d)
    Re = Re_delta(delta,sigma,gamma)
    return Re*gamma*N/S

# Generación de datos sintéticos
Trabajaremos con los infectados nuevos diarios, dado que esos son los datos que se miden en la realidad. El objetivo será tratar de reconstruir los parámetros originales. Truncaremos los datos para reducir el efecto de estabilización inicial. Partiremos con datos desde que los infectados activos superen los 100 casos. 
Por ahora despreciaremos los efectos del subreporte y del ruido de medición.

In [165]:
cfgfile = '../config_files/SEIR2.toml'
cfg = toml.load(cfgfile) # no es necesario, pero es util para ver el archivo de configuracion

In [172]:
beta = 0.4
I0 = 10
mu = 1
tE_I = 3
tI_R = 5
sigma = 1/tE_I
gamma = 1/tI_R

### Tiempo Serial
Uno de los mayores desafíos es tener un tiempo serial bien calculado.
Analíticamente la definición del tiempo serial es:  
$ T_s = T_{EI} + T_{IR} $  
Donde $T_{EI}$ y $T_{IR}$ son las medias de las distribuciones de tiempos de incubación e infecciosidad respectivamente. 
Para este análisis trabajaremos primero con los tiempos seriales de la misma simulación.

In [173]:
Ts = tE_I + tI_R

In [174]:
sims = cv19sim(cfg,beta = beta,mu=mu,I=I0,I_d=I0*beta,tE_I = tE_I,tI_R=tI_R,t_end=250)

In [175]:
sims.integrate()

In [176]:
plt.plot(sims.sims[0].I_d,label='Nuevos Infectados Diarios')
plt.legend(loc=0)
plt.show()

Se consideran los datos desde que hayan 100 casos

In [177]:
# Tiempo de los datos inicial
t0 = np.where(sims.sims[0].I >= 100)[0][0]
#t0 = 0

In [178]:
I_d = sims.sims[0].I_d[t0:]

### Cálculo R analítico
$R_e = \beta \frac{S}{N}T_{IR}$

In [179]:
Re_analitico = Re_analitico_calc(beta,gamma,sims.sims[0].S[t0:],sims.sims[0].N)

# Trabajo con la data sintética

In [180]:
plt.scatter(range(len(I_d)),I_d,label='New daily infected')
plt.title('Synthetic Data')
plt.show()

# Método 1: A. Bassi

In [14]:
# Calculo de delta
delta = getdelta(I_d)
# Calculo de Re
Re_delta = Re_delta_calc(delta,sigma,gamma)

In [15]:
fig, axs2 = plt.subplots(figsize=(10,6),linewidth=5,edgecolor='black',facecolor="white")

axs2.set_ylabel('Delta',color='tab:red')
axs2.plot(range(len(delta)),delta,label='delta',color='tab:red')

axs2.tick_params(axis='y', labelcolor='tab:red')

axs = axs2.twinx()
axs.plot(range(len(I_d)),I_d,color='tab:blue',label='Infectados nuevos diarios',linestyle='solid')
axs.set_yscale('log')
#axs.plot(sim.t,sim.I,color='tab:blue',label='Activos')
axs.set_ylabel('Infectados',color='tab:blue')
axs.tick_params(axis='y', labelcolor='tab:blue')

#axs2.set_xlim(0,200)
fig.suptitle('Delta e Id')
axs.legend(loc=1)
axs2.legend(loc=3)
fig.show()

# Método 2: Epyestim

In [16]:
# Format data, create synthetic dates
initdate = datetime(2020,3,1)
dates = [initdate+timedelta(days=int(j)) for j in sims.sims[0].t[t0:]]
cases = pd.Series(data=I_d,index=dates)

In [17]:
gt_distribution = np.zeros(50)

In [18]:
gt_distribution[Ts-1] = 1

In [19]:
def gt_distribution_continuous(sigma,gamma):
    pass

In [20]:
R_epistim_Ts = covid19.r_covid(cases,delay_distribution=np.array([1]),gt_distribution=gt_distribution,n_samples=300)['R_mean']

## Estudios de Re: Comparación R analítico, R delta y R delta aprox

In [21]:
fig, axs2 = plt.subplots(figsize=(10,6),linewidth=5,edgecolor='black',facecolor="white")

axs2.set_ylabel('Re',color='tab:red')
axs2.plot(range(len(Re_analitico[1:])),Re_analitico[1:],label='Analitico')
axs2.plot(range(len(Re_delta)),Re_delta,label='delta')
#axs2.plot(range(len(Re_delta_a)),Re_delta_a,label='delta aprox')
axs2.plot(range(len(R_epistim_Ts)),R_epistim_Ts,color='black',label='Epyestim',linestyle='dashed')
axs2.axhline(1,color='grey',linestyle='dashed')
axs2.tick_params(axis='y', labelcolor='tab:red')

axs = axs2.twinx()
axs.plot(range(len(I_d)),I_d,color='tab:blue',label='Diarios',linestyle='solid')
axs.set_yscale('log')
axs2.axvline(np.where(I_d==max(I_d))[0][0],color='grey',linestyle='dashed')
#axs.plot(sim.t,sim.I,color='tab:blue',label='Activos')
axs.set_ylabel('Infectados',color='tab:blue')
axs.tick_params(axis='y', labelcolor='tab:blue')


axs2.set_ylim(0,4)
fig.suptitle('Re e Id')
axs.legend(loc=1)
axs2.legend(loc=3)
fig.show()

plt.show()

## Calculo de beta
$ \beta(t) = R_e \frac{N}{S(t)} \gamma $

In [22]:
beta_Re_delta = gamma*Re_delta*sims.sims[0].N/sims.sims[0].S[t0:]
#beta_Re_delta_a = gamma*Re_delta_a*sims.sims[0].N/sims.sims[0].S[t0:]
beta_Re_analitico = gamma*Re_analitico*sims.sims[0].N/sims.sims[0].S[t0:]
beta_Re_epyestim_Ts = gamma*R_epistim_Ts*sims.sims[0].N/sims.sims[0].S[t0+7:]

In [23]:
fig, axs2 = plt.subplots(figsize=(10,6),linewidth=5,edgecolor='black',facecolor="white")


axs2.plot(range(len(Re_analitico[1:])),Re_analitico[1:],color = 'tab:red',label='Re analitico')
axs2.plot(range(len(Re_delta)),Re_delta,label='Re delta',color = 'tab:red',linestyle='dashed')
#axs2.plot(range(len(R_epistim_Ts)),R_epistim_Ts,color='tab:red',label='Epyestim',linestyle='dotted')
#axs2.plot(range(len(Re_delta_a)),Re_delta_a,label='delta aprox')
axs2.axhline(1,color='red',linestyle='dotted',alpha=0.5)


axs = axs2.twinx()
axs.plot(range(len(beta_Re_analitico)),beta_Re_analitico,label='Beta analitico',color = 'tab:blue')
axs.plot(range(len(beta_Re_delta)),beta_Re_delta,label='Beta delta',color = 'tab:blue',linestyle='dashed')


axs2.fill_between(range(len(Re_analitico[1:])), Re_analitico[1:], Re_delta[0:-1],color='tab:grey', alpha=0.4)

axs.axvline(np.where(beta_Re_delta==np.min(beta_Re_delta))[0][0],color='tab:blue',alpha=0.5,linestyle='dotted',label='Min beta')

axs2.plot(range(len(R_epistim_Ts)),R_epistim_Ts,color='tab:red',label='Epyestim',linestyle='dotted')
axs.plot(range(len(beta_Re_epyestim_Ts)),beta_Re_epyestim_Ts,label='beta Epyestim',color = 'tab:blue',linestyle='dotted')

axs2.tick_params(axis='y', labelcolor='tab:red',size=10)
axs.tick_params(axis='y', labelcolor='tab:blue',size=10)

axs2.set_ylabel('Re',color='tab:red',size=12)
axs.set_ylabel('Beta',color='tab:blue',size=12)

axs.set_ylim(0.3,0.6)
axs2.set_ylim(0,2.5)

fig.suptitle('Re y beta')

lines, labels = axs.get_legend_handles_labels()
lines2, labels2 = axs2.get_legend_handles_labels()
axs2.legend(lines + lines2, labels + labels2, loc=[0.05,0.1])
#axs.legend(loc=[0.05,0.05])
axs2.legend(loc=[0.05,0.17])
# Create a Rectangle patch
#rect = patches.Rectangle([.67,.24],.3,.4, linewidth=1, edgecolor='b', facecolor='white')

# Add the patch to the Axes
#axs.add_patch(rect)
ax_zoom = axs2.inset_axes([.67,.5,.3,.4])

init = 80
end = 120
ax_zoom.plot(np.arange(init,end),Re_analitico[init:end],color = 'tab:red',linestyle='solid')
ax_zoom.plot(np.arange(init,end),Re_delta[init:end],color = 'tab:red',linestyle='dashed')
ax_zoom.fill_between(np.arange(init,end),Re_analitico[init:end],Re_delta[init:end],color='tab:grey', alpha=0.2)
ax_zoom.axvline(np.where(beta_Re_delta==np.min(beta_Re_delta))[0][0],color='tab:blue',alpha=0.5,linestyle='dotted')

ax_zoom.set_title('Re difference',size=10)

if True:
   
    ax_zoom.plot(np.arange(init,end),R_epistim_Ts[init:end],color='tab:red',label='Epyestim',linestyle='dotted')
#ax_zoom.set_xticks([])
#ax_zoom.set_yticks([])
axs2.indicate_inset_zoom(ax_zoom)

fig.show()


In [181]:
fig, axs2 = plt.subplots(figsize=(10,6),linewidth=5,edgecolor='black',facecolor="white")


axs2.plot(range(len(Re_analitico[1:])),Re_analitico[1:],color = 'tab:red',label='Re analitico')
axs2.plot(range(len(Re_delta)),Re_delta,label='Re delta',color = 'tab:red',linestyle='dashed')
#axs2.plot(range(len(R_epistim_Ts)),R_epistim_Ts,color='tab:red',label='Epyestim',linestyle='dotted')
#axs2.plot(range(len(Re_delta_a)),Re_delta_a,label='delta aprox')
axs2.axhline(1,color='red',linestyle='dotted',alpha=0.5)


axs = axs2.twinx()
axs.plot(range(len(beta_Re_analitico)),beta_Re_analitico,label='Beta analitico',color = 'tab:blue')
axs.plot(range(len(beta_Re_delta)),beta_Re_delta,label='Beta delta',color = 'tab:blue',linestyle='dashed')
#axs.plot(range(len(beta_Re_epyestim_Ts)),beta_Re_epyestim_Ts,label='beta Epyestim',color = 'tab:blue',linestyle='dotted')

axs2.fill_between(range(len(Re_analitico[1:])), Re_analitico[1:], Re_delta[0:-1],color='tab:grey', alpha=0.4)

axs.axvline(np.where(beta_Re_delta==np.min(beta_Re_delta))[0][0],color='tab:blue',alpha=0.5,linestyle='dotted',label='Min beta')

axs2.tick_params(axis='y', labelcolor='tab:red',size=10)
axs.tick_params(axis='y', labelcolor='tab:blue',size=10)

axs2.set_ylabel('Re',color='tab:red',size=12)
axs.set_ylabel('Beta',color='tab:blue',size=12)

#axs.set_ylim(0.3,0.6)
#axs2.set_ylim(0,2.5)

fig.suptitle('Re y beta')

lines, labels = axs.get_legend_handles_labels()
lines2, labels2 = axs2.get_legend_handles_labels()
axs2.legend(lines + lines2, labels + labels2, loc=[0.05,0.1])
#axs.legend(loc=[0.05,0.05])
#axs2.legend(loc=[0.05,0.17])

ax_zoom = axs2.inset_axes([.67,.24,.3,.4])

init = 80
end = 120
ax_zoom.plot(np.arange(init,end),Re_analitico[init:end],color = 'tab:red',linestyle='solid')
ax_zoom.plot(np.arange(init,end),Re_delta[init:end],color = 'tab:red',linestyle='dashed')
ax_zoom.fill_between(np.arange(init,end),Re_analitico[init:end],Re_delta[init:end],color='tab:grey', alpha=0.2)
ax_zoom.axvline(np.where(beta_Re_delta==np.min(beta_Re_delta))[0][0],color='tab:blue',alpha=0.5,linestyle='dotted')

ax_zoom.set_title('Re difference',size=10)
#ax_zoom.set_xticks([])
#ax_zoom.set_yticks([])
axs2.indicate_inset_zoom(ax_zoom)

fig.show()


# Caracterización del error del cálculo de beta para diversos parámetros

In [151]:
beta2 = np.linspace(0.1,0.6,26)

In [33]:
tE_I2 = np.linspace(1,5,41)
tI_R2 = tE_I2
sigma2 = 1/tE_I2
gamma2 = 1/tI_R2

In [155]:
sims2 = cv19sim(cfg,beta = list(beta2),mu=mu,I=I0,I_d=10,tE_I = list(tE_I2),tI_R = 'tE_I',t_end=250) 

In [156]:
sims2.integrate()

### Calculos

In [157]:
I_d = []
delta2 = []
beta2_delta = []
beta2_error = []
for i in range(len(beta2)):
    aux = []
    auxdelta = []
    auxbeta = []
    auxerror = []
    for j in range(len(tE_I2)):
        Id_aux = sims2.sims[j][i].I_d
        betaaux = sims2.sims[j][i].beta(0)
        gamma_aux = 1/sims2.sims[j][i].tI_R(0)
        sigma_aux = 1/sims2.sims[j][i].tE_I(0)
        S_aux = sims2.sims[j][i].S
        N_aux = sims2.sims[j][i].N
        delta_aux = getdelta(Id_aux)
        betaaux_delta = beta_delta(delta_aux,S_aux,N_aux,gamma_aux,sigma_aux)
        auxdelta.append(delta_aux)
        auxbeta.append(betaaux_delta)
        auxerror.append(((betaaux_delta - betaaux)**2).mean())
    delta2.append(auxdelta)
    I_d.append(Id_aux)
    beta2_delta.append(auxbeta)
    beta2_error.append(auxerror)


In [158]:
len(beta2)

26

In [82]:
np.shape(beta2_delta)

(26, 41, 250)

In [161]:
# Contour Plot
err = beta2_error
err_max = np.max(err) 
levels = np.arange(0,err_max*1.1,0.001)
fig,ax=plt.subplots(1,1)
cp = ax.contourf(tE_I2,beta2,err,levels)
cp2 = ax.contour(tE_I2,beta2,err,[0,0.005,err_max],colors='white',linestyles='dashed')
#cp2 = ax.contour(tE_I_s,tI_R_s,err,[0,np.min(err)+0.0000001,err_max],colors='white',linestyles='dotted')

fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Error obtaining beta from delta')
ax.set_xlabel('Infectious time')
ax.set_ylabel('Beta')
plt.show()

In [96]:
np.shape(beta2_delta)

(26, 41, 250)

In [138]:
i = 14

for j in range(int(len(tE_I2)/2)):
    plt.plot(beta2_delta[i][2*j],label='tE_I: '+"{0:.3g}".format(tE_I2[2*j])+' days')
plt.axhline(beta2[i])
plt.title('Beta: '+str(beta2[i]))
plt.legend(loc=0)
plt.show()

In [130]:
from matplotlib import cm

In [131]:
colors = plt.cm.rainbow_r(np.linspace(0,1,int(len(beta2)/2+1)))

In [186]:
j = 20
for i in range(int(len(beta2)/2)):
    plt.plot(beta2_delta[i][2*j],label='beta: '+"{0:.3g}".format(beta2[i]),color=colors[i])
    plt.axhline(beta2[i],linestyle='dashed',color=colors[i])
plt.title('tE_I: '+str(tE_I2[2*j]))
plt.legend(loc=0)
plt.show()

In [185]:
# I
j = 20
for i in range(int(len(beta2)/2)):
    plt.plot(sims2.sims[2*j][i].I_d,label='beta: '+"{0:.3g}".format(sims2.sims[2*j][i].beta(0)),color=colors[i])
    #plt.axhline(beta2[i],linestyle='dashed',color=colors[i])
plt.title('tE_I: '+str(tE_I2[2*j]))
plt.legend(loc=0)
plt.show()

In [181]:
j = 20

fig, axs2 = plt.subplots(figsize=(10,6),linewidth=5,edgecolor='black',facecolor="white")

for i in range(int(len(beta2)/2)):
    axs2.plot(beta2_delta[i][2*j],label='beta: '+"{0:.3g}".format(beta2[i]),color=colors[i])
    axs2.axhline(beta2[i],linestyle='dashed',color=colors[i])


axs = axs2.twinx()


for i in range(int(len(beta2)/2)):
    plt.plot(sims2.sims[2*j][i].I_d,label='beta: '+"{0:.3g}".format(sims2.sims[2*j][i].beta(0)),color=colors[i])
    #plt.axhline(beta2[i],linestyle='dashed',color=colors[i])
plt.title('tE_I: '+str(tE_I2[2*j]))
plt.legend(loc=0)
plt.show()

axs.plot(range(len(beta_Re_analitico)),beta_Re_analitico,label='Beta analitico',color = 'tab:blue')
axs.plot(range(len(beta_Re_delta)),beta_Re_delta,label='Beta delta',color = 'tab:blue',linestyle='dashed')
#axs.plot(range(len(beta_Re_epyestim_Ts)),beta_Re_epyestim_Ts,label='beta Epyestim',color = 'tab:blue',linestyle='dotted')

axs2.fill_between(range(len(Re_analitico[1:])), Re_analitico[1:], Re_delta[0:-1],color='tab:grey', alpha=0.4)

axs.axvline(np.where(beta_Re_delta==np.min(beta_Re_delta))[0][0],color='tab:blue',alpha=0.5,linestyle='dotted',label='Min beta')

axs2.tick_params(axis='y', labelcolor='tab:red',size=10)
axs.tick_params(axis='y', labelcolor='tab:blue',size=10)

axs2.set_ylabel('Re',color='tab:red',size=12)
axs.set_ylabel('Beta',color='tab:blue',size=12)

#axs.set_ylim(0.3,0.6)
#axs2.set_ylim(0,2.5)

fig.suptitle('tE_I: '+str(tE_I2[2*j]))

lines, labels = axs.get_legend_handles_labels()
lines2, labels2 = axs2.get_legend_handles_labels()
axs2.legend(lines + lines2, labels + labels2, loc=[0.05,0.1])
#axs.legend(loc=[0.05,0.05])
#axs2.legend(loc=[0.05,0.17])

ax_zoom = axs2.inset_axes([.67,.24,.3,.4])

init = 80
end = 120
ax_zoom.plot(np.arange(init,end),Re_analitico[init:end],color = 'tab:red',linestyle='solid')
ax_zoom.plot(np.arange(init,end),Re_delta[init:end],color = 'tab:red',linestyle='dashed')
ax_zoom.fill_between(np.arange(init,end),Re_analitico[init:end],Re_delta[init:end],color='tab:grey', alpha=0.2)
ax_zoom.axvline(np.where(beta_Re_delta==np.min(beta_Re_delta))[0][0],color='tab:blue',alpha=0.5,linestyle='dotted')

ax_zoom.set_title('Re difference',size=10)
#ax_zoom.set_xticks([])
#ax_zoom.set_yticks([])
axs2.indicate_inset_zoom(ax_zoom)

fig.show()


In [154]:
sims2.sims[0][0].I_d[0]

3000.0

In [140]:
plt.plot(sims2.sims[0][0].I_d)

In [142]:
sims2.sims[0][0].beta(0)

0.1

In [113]:
10%8

2

In [120]:
fig, axs = plt.subplots(3,4,figsize=(10,6),linewidth=5,edgecolor='black',facecolor="white")

for i in range(12):
    for j in range(int(len(tE_I2)/2)):
        axs[i%3,i%4].plot(beta2_delta[2*i][2*j],label='tE_I: '+"{0:.3g}".format(tE_I2[2*j])+' days')
        axs[i%3,i%4].axhline(beta2[2*i])
        axs[i%3,i%4].set_title('Beta: '+str(beta2[2*i]))
        axs[i%3,i%4].legend(loc=0)
fig.show()

# Análisis de sensibilidad de tiempo serial
Para hacer este estudio utilizaremos los mismos datos sintéticos y calcularemos el Re y beta utilizando distintos gammas y sigmas para ver como crece el error a medida que nos alejamos de los valores correctos

In [28]:
tE_I_s = np.arange(2,6,0.1)
tI_R_s = np.arange(2,6,0.1)
Ts_s = np.array([[i+j for i in tE_I_s] for j in tI_R_s])

In [29]:
sigma_s = 1/tE_I_s
gamma_s = 1/tI_R_s

### MSE y RMSE Re vs Re_analitico
Re calculado para las distintas combinaciones de $\sigma$ y $\gamma$

In [ ]:
P = 1
# R delta
Re_delta_s_a = np.zeros(np.shape(Ts_s), dtype=object)  
Re_delta_s = np.zeros(np.shape(Ts_s), dtype=object)
error_s = np.zeros(np.shape(Ts_s))
error_s_a = np.zeros(np.shape(Ts_s))
for i in range(len(sigma_s)):
    for j in range(len(gamma_s)):
        Re_delta_s[i,j] = Re_delta_calc(delta,sigma_s[i],gamma_s[j])
        Re_delta_s_a[i,j] = Re_delta_a_calc(delta,sigma_s[i],gamma_s[j])

        error_s[i,j] = ((P*Re_delta_s[i,j] - P*Re_analitico[0:])**2).mean()
        error_s_a[i,j] = ((P*Re_delta_s_a[i,j] - P*Re_analitico[0:])**2).mean()            

In [ ]:
# Contour Plot
err =error_s
err_max = np.max(err) 
levels = np.arange(0,err_max*1.1,0.001)
fig,ax=plt.subplots(1,1)
cp = ax.contourf(tE_I_s,tI_R_s,err,levels)

cp2 = ax.contour(tE_I_s,tI_R_s,err,[0,err_max*0.25,err_max],colors='white',linestyles='dashed')
cp2 = ax.contour(tE_I_s,tI_R_s,err,[0,np.min(err)+0.0000001,err_max],colors='white',linestyles='dotted')

fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('MSE vs Re analítico')
ax.set_xlabel('Incubation period')
ax.set_ylabel('Infectious period')
plt.show()

In [ ]:
# Contour Plot
err = np.sqrt(error_s)
err_max = np.max(err) 
levels = np.arange(0,err_max*1.1,0.001)
fig,ax=plt.subplots(1,1)
cp = ax.contourf(tE_I_s,tI_R_s,err,levels)

cp2 = ax.contour(tE_I_s,tI_R_s,err,[0,err_max*0.25,err_max],colors='white',linestyles='dashed')
cp2 = ax.contour(tE_I_s,tI_R_s,err,[0,np.min(err)+0.0000001,err_max],colors='white',linestyles='dotted')

fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('RMSE vs Re analítico')
ax.set_xlabel('Incubation period')
ax.set_ylabel('Infectious period')
plt.show()

### RMSE Re vs Re_delta
Re calculado para las distintas combinaciones de $\sigma$ y $\gamma$ vs el Re calculado con $\delta$ para los valores con los que se hizo la simulación original

In [ ]:
P = 1
# R delta
Re_delta_s_a = np.zeros(np.shape(Ts_s), dtype=object)  
Re_delta_s = np.zeros(np.shape(Ts_s), dtype=object)
error_s_d = np.zeros(np.shape(Ts_s))
error_s_a_d = np.zeros(np.shape(Ts_s))
for i in range(len(sigma_s)):
    for j in range(len(gamma_s)):
        Re_delta_s[i,j] = Re_delta_calc(delta,sigma_s[i],gamma_s[j])
        Re_delta_s_a[i,j] = Re_delta_a_calc(delta,sigma_s[i],gamma_s[j])        

        error_s_d[i,j] = ((P*Re_delta_s[i,j] - P*Re_delta[:])**2).mean()
        error_s_a_d[i,j] = ((P*Re_delta_s_a[i,j] - P*Re_delta[:])**2).mean()
            

In [ ]:
# Contour Plot
err =error_s_d
err_max = np.max(err) 
levels = np.arange(0,err_max*1.1,0.001)
fig,ax=plt.subplots(1,1)
cp = ax.contourf(tE_I_s,tI_R_s,err,levels)

cp2 = ax.contour(tE_I_s,tI_R_s,err,[0,err_max*0.25,err_max],colors='white',linestyles='dashed')
cp2 = ax.contour(tE_I_s,tI_R_s,err,[0,np.min(err)+0.0000001,err_max],colors='white',linestyles='dotted')

fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('MSE vs Re delta')
ax.set_xlabel('Incubation period')
ax.set_ylabel('Infectious period')
plt.show()

In [ ]:
# Contour Plot
err = np.sqrt(error_s_d)
err_max = np.max(err) 
levels = np.arange(0,err_max*1.1,0.001)
fig,ax=plt.subplots(1,1)
cp = ax.contourf(tE_I_s,tI_R_s,err,levels)

cp2 = ax.contour(tE_I_s,tI_R_s,err,[0,err_max*0.25,err_max],colors='white',linestyles='dashed')
cp2 = ax.contour(tE_I_s,tI_R_s,err,[0,np.min(err)+0.0000001,err_max],colors='white',linestyles='dotted')

fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('RMSE vs Re delta')
ax.set_xlabel('Incubation period')
ax.set_ylabel('Infectious period')
plt.show()

# Error cálculo Re para distintos R0

In [ ]:
beta2 = list(np.arange(0.2,0.6,0.01))
R0 = np.array(beta2)*tI_R

In [ ]:
sims2 = cv19sim(cfg,beta = beta2,mu=mu,I=I0,I_d=I0*beta,tE_I = tE_I,tI_R=tI_R,t_end=250)
sims2.integrate()

### Calculo de R por delta

In [ ]:
t0=1

In [ ]:
Re_analitico2 = [beta2[i]*sims2.sims[i].S[t0:-1]/sims2.sims[i].N*tI_R for i in range(len(beta2))]


In [ ]:
delta2 = [getdelta(sims2.sims[i].I_d[t0:]) for i in range(len(beta2))]

In [ ]:
# R delta
Re_delta2 = []
for j in delta2:
    Re_delta2.append(Re_delta_calc(j,sigma,gamma))

Re_delta2=np.array(Re_delta2)

# R delta aprox
#Re_delta_a = 1 + delta*Ts

In [ ]:
# Error
err2 = []
for i in range(len(beta2)):
    err2.append(np.sqrt((Re_analitico2[i][0:100]-Re_delta2[i][0:100]).mean()))

In [ ]:
plt.plot(R0,err2)
plt.title('RMSE R vs R0')
plt.xlabel('R0')
plt.ylabel('RMSE')
plt.plot()

# Error cálculo Re para distintos R0 y Ts

In [ ]:
beta3 = list(np.arange(0.2,0.6,0.01))
R0 = np.array(beta2)*tI_R
Ts3 = np.arange(1,6,0.1)

In [ ]:
sims3 = []
for i in Ts3:
    sims3.append(cv19sim(cfg,beta = beta2,mu=mu,I=I0,I_d=I0*beta,tE_I = i/2,tI_R=i/2,t_end=250))

for i in range(len(Ts3)):
    sims3[i].integrate()

In [ ]:
sims = np.array([i.sims for i in sims3])

In [ ]:
t0 = 10

In [ ]:
Re_analitico3 = [[sims[i,j].beta(0)*sims[i,j].S[t0:-1]/sims[i,j].N*sims[i,j].tI_R(0) for j in range(len(beta3))]for i in range(len(Ts3))]

In [ ]:
delta3 = [[getdelta(sims[i,j].I_d[t0:-1]) for j in range(len(beta3))]for i in range(len(Ts3))]

In [ ]:
i = 40
j = 39
aux = delta3[i][j]

In [ ]:
plt.plot(sims[i,j].I_d)

In [ ]:
plt.plot(range(len(aux)),aux)

In [ ]:
# R delta
Re_delta3 = []
for j in delta2:
    aux = []
    for i in range(len(j)):
        aux.append(1 + j[i]*Ts + sigma*gamma/(sigma + gamma)*(j[i]*Ts)**2)
    Re_delta2.append(aux)
Re_delta2=np.array(Re_delta2)

# R delta aprox
#Re_delta_a = 1 + delta*Ts

# To Do:
[ ] Comprobar el orden de las variables en los arreglos sigma-gamma  
[ ] Verificar el calculo de R porque no me están calzando bien los R con las magnitudes de los otros.
[ ] Grid-plot  


[ ] Estudiar cuanto aumenta el error a medida que cambian el tiempo serial, gamma y sigma 

[x] Calcular Beta desde el R  
[ ] Reconstruir las curvas con los parámetros obtenidos  
[ ] Tratar de compatibilizar los cálculos de R de Felipe y de la nueva librería, y compararlos con el cálculo analítico  

[ ] Revisar calculo del error entre los cálculos de R para el contourplot.  
[ ] Revisar calculo del error entre los cálculos de R. 